In [71]:
from keras.models import model_from_json
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(128,kernel_size=(3,3),input_shape=(64,64,3), activation='relu'))


model.add(layers.Conv2D(256,kernel_size=(3,3), activation='relu'))


model.add(layers.Conv2D(256,kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(256,kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()
# model.load_weights("normal_cnn.h5")



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 60, 60, 256)       295168    
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 58, 58, 256)       590080    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 29, 29, 256)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 27, 27, 256)       590080    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 43264)            

In [72]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True    
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)



train_generator = train_datagen.flow_from_directory(
    'v1/v1/train',
    target_size = (64,64),
    batch_size = 16,
    class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
    'v1/v1/validation',
    target_size = (64,64),
    batch_size = 4,
    class_mode = 'binary'
)
test_generator = test_datagen.flow_from_directory(
    'v1/v1/test',
    batch_size=4,
    target_size=(64,64),
    class_mode ='binary'
)




Found 1005 images belonging to 2 classes.
Found 514 images belonging to 2 classes.
Found 305 images belonging to 2 classes.


In [75]:

model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy'])


In [76]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=600,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=30
)

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_17/convolution (defined at C:\Users\Rentalhub\anaconda3\envs\joong\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_8839]

Function call stack:
keras_scratch_graph


In [59]:
import cv2

face_classifier = cv2.CascadeClassifier(r'C:\Users\Rentalhub\anaconda3\envs\lim\Library\etc\haarcascades\haarcascade_frontalface_default.xml')

def face_detector(img, size = 0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    #detectMultiScale = input이미지에서 object 검출, 여기선 얼굴 검출로 초기화 했기 때문에 얼굴 검출함
    
    #찾은 얼굴이 없으면 그냥 사진 그대로 리턴.
    if faces is() :
        return img,[]
    for (x,y,w,h) in faces :
        cv2.rectangle(img,(x,y),(x+w, y+h), (0,255,0),1)
        roi = img[y:y+h, x:x+w]
        print(roi.shape)
        print(type(roi))
#         roi = cv2.resize(roi,(64,64))
        roi = cv2.resize(roi, dsize = (64,64), interpolation = cv2.INTER_AREA)
        print(roi.shape)
        print(type(roi))
        roi = roi.astype('float')/255
        roi = roi.reshape((1,64,64,3))
#         print(roi.shape)
    #img = 사진 안 얼굴 위에 사각형이 그려저 있는 사진
    #roi = 사각형 모양으로 잘라 진 사진
    return img, roi 

In [60]:
cap = cv2.VideoCapture(0)

while True:
    #frame읽기에 성공하면 ret에 true, 실패하면 false가 저장됨.
    #frame에는 읽어온 frame(이미지)가 저장
    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    print("point 1")
    print(frame.shape)
    try:
        print("point 2")
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        print("point 3")
        result = model.predict(face,step=1)
        print("point 4")
        #result[0][0]는 0에 가까울 수록 내 얼굴임
        #고로 confidence가 높을 수록 내 얼굴 ㅎ.,ㅎ
        confidence = 100 * (1 - result[0][0] )
        print(confidence)
        print(result[0][0])
        
        display_string = str(confidence) + "% similar with My face"
        cv2.putText(image, display_string, (100,120), cv2.FONT_HERSHEY_COMPLEX, 1, (100, 100, 100),2)
        
        if confidence > 80 :
            cv2.putText(image, "Lim Joongsub", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0),2)
        cv2.imshow('face',image)
        
    except:
        cv2.putText(image, "No face", (100,120), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0),2)
        cv2.imshow('face',image)
    
    #1은 enter
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()
       

(162, 162, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
point 1
(480, 640, 3)
point 2
point 1
(480, 640, 3)
point 2
point 1
(480, 640, 3)
point 2
point 1
(480, 640, 3)
point 2
point 1
(480, 640, 3)
point 2
(151, 151, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(164, 164, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(159, 159, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(165, 165, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(151, 151, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(147, 147, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(161, 161, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(166, 166, 3)
<cla

(136, 136, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(134, 134, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(130, 130, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(130, 130, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(126, 126, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(126, 126, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(128, 128, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(129, 129, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(129, 129, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.ndarray'>
point 1
(480, 640, 3)
point 2
(129, 129, 3)
<class 'numpy.ndarray'>
(64, 64, 3)
<class 'numpy.

In [61]:
result = model.predict(face)
print(result)

# plt.imshow(face)
# print(face.shape)
# face = cv2.resize(face, dsize = (64,64), interpolation = cv2.INTER_AREA)
# face = face.astype('float')/255
# face = face.reshape((1,64,64,3))

# plt.imshow(face)
# plt.show(face)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_13/convolution (defined at C:\Users\Rentalhub\anaconda3\envs\joong\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_1832]

Function call stack:
keras_scratch_graph
